# E-commerce Brasileiro (Olist) - Classificação

## 1. Tratamento e Preparação de Dados

### 1.1. Organização e Carga

In [1]:
!pip install imbalanced-learn


   ---------------------------------------- 0/2 [sklearn-compat]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   -------------------- ------------------- 1/2 [imbalanced-learn]
   ---------------------------------------- 2/2 [imbalanced-learn]




[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

path = '../../data/raw/'

orders = pd.read_csv(f'{path}olist_orders_dataset.csv')
items = pd.read_csv(f'{path}olist_order_items_dataset.csv')
reviews = pd.read_csv(f'{path}olist_order_reviews_dataset.csv')
products = pd.read_csv(f'{path}olist_products_dataset.csv')

### 1.2. Merge

In [3]:
# Para saber as datas de cada avaliação
df_class = pd.merge(reviews, orders, on='order_id', how='inner')

# Para saber preço e frete
df_class = pd.merge(df_class, items, on='order_id', how='inner')

# Para saber categoria e peso
df_class = pd.merge(df_class, products, on='product_id', how='inner')

print(f"Total de linhas para processar: {df_class.shape[0]}")

Total de linhas para processar: 112372


### 1.3. Target

Se tentarmos prever notas de 1 a 5, o modelo vai ter dificuldade porque a maioria das pessoas dá nota 5. Vamos simplificar para um problema binário: Satisfeito (1) vs Insatisfeito (0).

In [4]:
# Transformando datas para o formato correto
date_cols = ['order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']
for col in date_cols:
    df_class[col] = pd.to_datetime(df_class[col])

# Feature: O produto atrasou? (1 = Sim, 0 = Não)
# Comparamos a data de entrega real com a estimada pelo site
df_class['atrasado'] = (df_class['order_delivered_customer_date'] > df_class['order_estimated_delivery_date']).astype(int)

# Nota 4 e 5 = 1 (Satisfeito) | Nota 1, 2 e 3 = 0 (Insatisfeito)
df_class['target'] = df_class['review_score'].apply(lambda x: 1 if x >= 4 else 0)

### 1.4. Limpeza e Features

Alguns pedidos podem não ter sido entregues ainda (data de entrega vazia), e alguns produtos podem estar sem peso ou tamanho cadastrado.

In [5]:
# removendo linhas onde a entrega não aconteceu
df_class = df_class.dropna(subset=['order_delivered_customer_date'])

# colunas para o modelo: Preço, Frete, Atributos do Produto e o Atraso
features_list = [
    'price', 
    'freight_value', 
    'product_weight_g', 
    'product_length_cm', 
    'product_height_cm', 
    'product_width_cm', 
    'atrasado'
]

# criando matriz X (dados) e vetor y (alvo)
X = df_class[features_list].copy()
y = df_class['target']

# tratando nulos restantes
X = X.fillna(X.median())

print(f"Formato do X: {X.shape}")

Formato do X: (110012, 7)


### 1.5. Divisão de Treino/Teste e Escalonamento

Padronizando dados para modelos sensíveis à escala, como KNN e MLP.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Divisão em 80% treino e 20% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Padronização (Z-score)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Atrasado: feature "coringa". Estatisticamente, é o que mais impacta a nota.

Fillna(median): mais robusta a outliers do que a média.

StandardScaler: Essencial para que o modelo não ache que "Peso" é mais importante que "Preço" só porque os números do peso são maiores (ex: 1000g vs 50 reais).

### 1.6. Balanceamento

O SMOTE cria "exemplos sintéticos" da classe minoritária (insatisfeitos) para que o modelo tenha a mesma quantidade de exemplos de ambos os lados para aprender.

In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter

print(f"Distribuição antes do SMOTE: {Counter(y_train)}")

# Aplicando o SMOTE nos dados de TREINO
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_scaled, y_train)

print(f"Distribuição após o SMOTE: {Counter(y_train_res)}")

Distribuição antes do SMOTE: Counter({1: 67566, 0: 20443})
Distribuição após o SMOTE: Counter({1: 67566, 0: 67566})


## 2. Modelos

fazendo uso do GridSearchCV para validação cruzada

### 2.1. Modelo 1: KNN (K-Nearest Neighbors)

Jusificativa: Modelo não-paramétrico baseado em distância. Assume que clientes com experiência de compra similares (preço, frete e atraso parecidos) tendem a dar a mesma nota.

Hiperparâmetro: Ajustamos o n_neighbors para evitar overfitting ou generalização excessiva.

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

knn = KNeighborsClassifier()

# definindo hiperparâmetros
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

# GridSearchCV com Validação Cruzada (cv=5)
grid_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='f1', n_jobs=-1)

grid_knn.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros para o KNN: {grid_knn.best_params_}")

Melhores parâmetros para o KNN: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}


### 2.2. Modelo 2: Regressão Logística

Justificativa: Classificador Linear clássico. Verifica se o problema de satisfação do cliente pode ser resolvido com uma simples separação linear entre classes.

Hiperparâmetro: ajuste de parâmetro C para que o modelo não decore demais os dados.

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', random_state=42)

# C menor = regularização mais forte
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

grid_lr = GridSearchCV(lr, param_grid_lr, cv=5, scoring='f1', n_jobs=-1)

grid_lr.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros LR: {grid_lr.best_params_}")

Melhores parâmetros LR: {'C': 0.1, 'penalty': 'l1'}


### 2.3. Modelo 3: Árvore de Decisão

Justificativa: Modelo não-linear e interpretável. Captura regras de negócio como: Se o frete > X E atraso = 1, então a nota é 0.

Hiperparâmetro: Controle da max_depth para evitar que a árvore cresça indefinidamente.

In [10]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)

param_grid_dt = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='f1', n_jobs=-1)

grid_dt.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros Árvore: {grid_dt.best_params_}")

Melhores parâmetros Árvore: {'criterion': 'entropy', 'max_depth': None, 'min_samples_split': 2}


F1-Score é a métrica mais honesta, pois ela faz um balanço entre a Precisão (não errar quando diz que é insatisfeito) e o Recall (não deixar passar nenhum insatisfeito).

### 2.4. Modelo 4: Random Forest

Justificativa: Combina várias áreas de decisão para reduzir a variância. Um dos modelos mais robustos para dados tabulares, lidando bem com complexidade e interações entre variáveis.

Hiperparâmetro: Teste do n_estimators para equilibrar performance e custo computacional.

In [11]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)

# hiperparametros
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'criterion': ['gini', 'entropy']
}

grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='f1', n_jobs=-1)

grid_rf.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros Random Forest: {grid_rf.best_params_}")

Melhores parâmetros Random Forest: {'criterion': 'entropy', 'max_depth': None, 'n_estimators': 200}


### 2.5. Modelo: MLP

Justificativa: Representa as Redes Neurais. Modelo escolhido pois captura padrões matemáticos complexos e não-lineares que modelos mais simples podem ignorar.

Hiperparâmetros: Ajuste da hidden_layer_sizes para definir a capacidade de processamento da rede.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=500, random_state=42)

# Hiperparâmetros
param_grid_mlp = {
    'hidden_layer_sizes': [(50,), (100,), (50, 20)],
    'activation': ['tanh', 'relu'],
    'learning_rate_init': [0.001, 0.01]
}

grid_mlp = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='f1', n_jobs=-1)

grid_mlp.fit(X_train_res, y_train_res)

print(f"Melhores parâmetros MLP: {grid_mlp.best_params_}")